## Tips

気づいたこと、気になったこと、間違いやすいことなどをまとめる


## print(type(object)) と type(object) 関数の表示の違い

In [ ]:
list_1 = list(range(5))
print('type(list_1):', type(list_1))
print('type(1):', type(1))
print('type(\'characters\'):', type('characters'))
print('type({\'1st\':1, \'2nd\':2, \'3rd\':3}):', type({'1st':1, '2nd':2, '3rd':3}))
print('type({\'1st\':1, \'2nd\':2, \'3rd\':3}.keys()):', type({'1st':1, '2nd':2, '3rd':3}.keys()))

In [ ]:
type(list({'1st':1, '2nd':2, '3rd':3}.keys()))

この表示の違いは、<br>
オブジェクト内のオブジェクト自体のクラスを格納している変数の名前の文字列を単に表示しているか<br>
それとも<br>
オブジェクト内のオブジェクト自体のクラスを格納している変数を<br>
「 print 関数の書式付き印字によって整型して」表示しているかの違い

In [ ]:
i = 1
print(i.__class__.__name__) # オブジェクト i が属しているクラスの名前の文字列
print(i.__class__)                # オブジェクト i が属しているクラス自体が、print関数により "<class 'クラスの名前'>" という形式に整型される
print('%.3f %04d'% (i, i))     # 書式付き印字

l = list({'1st':1, '2nd':2, '3rd':3}.keys())
print('%s'% l.__class__.__name__)
print('%s'% l.__class__)

## listのコンストラクタ引数
リストクラスをコンストラクタで作成する場合は、<br>
コンストラクタの引数に、iterable を指定しないといけない。

In [ ]:
list_1 = list([1, 2, 3])                     # リストが作成される
print( list_1 )

try:
    list_2 = list(2)                          # リストが作成されず、TypeError: 'int' object is not iterable が発生する
except TypeError as e:
    print(e.__class__.__name__,   e) # リストクラスをコンストラクタで作成する場合は、iterable（イテラブル）な引数を指定しなければならない

list_3 = [1, 2, 3]                           # 暗黙的に list() が実行されている → list()コンストラクタが内部で実行されているのと同じ
print( list_3 )


## dict の item を定義したときの順序は保持されるのか
---
Pythonでdictを実装する上では、メモリとキーの探索の効率化のため、順序は保持されています。<br>
ただし、dict内の定義順をPython内部から参照することは不能です。<br>
<br>
<u>また、Pythonのバージョンによっては初期化時の順序が保持されないこともあると思われますので、<br>
初期化時の順序に依存しないプログラムを書くことが肝要です。</u>

In [ ]:
dict_1 = dict(apple = 167,  cherry = 578,  banana = 262)
for key in dict_1.keys():
    print('[key] %s : [value] %d'% (key, dict_1[key]))

### dict クラスの深堀
dictのコンストラクタに渡す引数が、そもそも辞書になっている。

- python.orgによるdictの説明<br>
  - [dict](https://docs.python.org/ja/3/library/stdtypes.html#dict)

- オブジェクトの \__dict__属性に辞書を設定して、組み込み関数 vars() に渡すと dict を作ってくれる<br>
```Python
    def __init__(self, **kwargs):
        for name in kwargs:
            setattr(self, name, kwargs[name])
```
  クラスのインスタンス初期化時に可変引数を名前付きパラメータの名前とその値に分解して、settattr組み込み関数でobjectの属性（attribute）として設定する。

- 組み込み関数 vars()とは ─  build-in function vars()<br>
 \_\_dict\_\_ 属性を持つオブジェクトの\_\_dict\_\_ 属性を返します。
  - [vars](https://docs.python.org/ja/3/library/functions.html#vars)
- \_\_dict\_\_属性とは<br>
  オブジェクトの (書き込み可能な) 属性を保存するために使われる辞書またはその他のマッピングオブジェクトです。
  - [object.\_\_dict\_\_](https://docs.python.org/ja/3/library/stdtypes.html#object.__dict__)

In [ ]:
dict_1 = dict(apple = 167,  cherry = 578,  banana = 262)
print( dict_1 )
print(dict_1.__class__.__name__)

# オブジェクトにsetattr()関数で値を設定しさえすれば辞書が作成できる
class mydict:
    pass          # 何もしないクラス
dict_me = mydict()
setattr(dict_me, 'apple', 167)
setattr(dict_me, 'cherry', 578)
setattr(dict_me, 'banana', 262)
dict_2 = vars(dict_me)
print(dict_2)
print(dict_2.__class__.__name__)

### Cpythonでのdictの実装
　密なハッシュ値のテーブルを参照する疎なインデックステーブルというデータ構造で管理されており、疎なインデックステーブルには辞書にキーと値を挿入したときの順番が保持されている。

- python dict source  code ─ dictはpythonでどのように実現されているのか<br>
  - [cpython](https://github.com/python/cpython/blob/5ab745fc51e159ead28b523414e52f0bcc1ef353/Objects/dictobject.c#L1)
  - [insertion order](https://github.com/python/cpython/blob/5ab745fc51e159ead28b523414e52f0bcc1ef353/Objects/dictobject.c#L89)
```
Preserving insertion order
...(An omission)
To preserve the order in a split table, a bit vector is used  to record the
insertion order. When a key is inserted the bit vector is shifted up by 4 bits
and the index of the key is stored in the low 4 bits.
As a consequence of this, split keys have a maximum size of 16.
```

  - dk_indices
    - [dictkeys_set_index](https://github.com/python/cpython/blob/5ab745fc51e159ead28b523414e52f0bcc1ef353/Objects/dictobject.c#L351)

- dictnotes.txt ─辞書の設計と最適化を調査するときの説明と辞書の典型的な用途、チューニングのパラメータ、できうる最適化の着想<br>
  -  [dictnotes.txt](https://github.com/python/cpython/blob/e466faa9df9a1bd377d9725de5484471bc4af8d0/Objects/dictnotes.txt#L1)
  - Data Layout
```
Dictionaries are composed of 3 components:
The dictobject struct itself
A dict-keys object (keys & hashes)
A values array
```

- dictionary のハッシュテーブルの基本アイディア
  - [Python-Dev](https://mail.python.org/pipermail/python-dev/2012-December/123028.html)
  - 密なハッシュ値のテーブルを参照する疎なテーブルというデータ構造で、メモリ効率と反復（イテレーション）の高速化
```
indices =  [None, 1, None, None, None, 0, None, 2]
entries =  [[-9092791511155847987, 'timmy', 'red'],
                [-8522787127447073495, 'barry', 'green'],
                [-6480567542315338377, 'guido', 'blue']]
```

- 「PyPyにおける高速なメモリ効率的で秩序だった辞書」─PyPyはPythonの高速実装<br>
  - [Faster, more memory efficient and more ordered dictionaries on PyPy](https://morepypy.blogspot.com/2015/01/faster-more-memory-efficient-and-more.html)